In [1]:
import requests # 웹 페이지 소스를 얻기 위한 패키지(기본 내장 패키지이다.)
from bs4 import BeautifulSoup # 웹 페이지 소스를 얻기 위한 패키지, 더 간단히 얻을 수 있다는 장점이 있다고 한다.
from datetime import datetime                                # (!pip install beautifulsoup4 으로 다운받을 수 있다.)
import pandas as pd # 데이터를 처리하기 위한 가장 기본적인 패키지
import time # 사이트를 불러올 때, 작업 지연시간을 지정해주기 위한 패키지이다. (사이트가 늦게 켜지면 에러가 발생하기 때문)
import urllib.request #
from selenium.webdriver import Chrome
import json
import re     
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt

In [2]:
comment_data = pd.read_csv('기성용.csv')

In [3]:
comment_data

Video ID                                    Title  \
0     rFknOnjBbvg             기성용, 구자철, 황희찬, 김민재, 백승호 영상통화   
1     rFknOnjBbvg             기성용, 구자철, 황희찬, 김민재, 백승호 영상통화   
2     rFknOnjBbvg             기성용, 구자철, 황희찬, 김민재, 백승호 영상통화   
3     rFknOnjBbvg             기성용, 구자철, 황희찬, 김민재, 백승호 영상통화   
4     rFknOnjBbvg             기성용, 구자철, 황희찬, 김민재, 백승호 영상통화   
...           ...                                      ...   
2355  GxxgbkVnluQ  [트랜스퍼H] 마요르카 떠난 기성용, 전북 - 서울 K리그행 가능성은?   
2356  GxxgbkVnluQ  [트랜스퍼H] 마요르카 떠난 기성용, 전북 - 서울 K리그행 가능성은?   
2357  GxxgbkVnluQ  [트랜스퍼H] 마요르카 떠난 기성용, 전북 - 서울 K리그행 가능성은?   
2358  GxxgbkVnluQ  [트랜스퍼H] 마요르카 떠난 기성용, 전북 - 서울 K리그행 가능성은?   
2359  GxxgbkVnluQ  [트랜스퍼H] 마요르카 떠난 기성용, 전북 - 서울 K리그행 가능성은?   

                                                Comment  
0     자철 선수가 매개체 되어 해외에서 생활하는 선수들과 꾸준히 소통하고 상호 정보 공유...  
1              성용 선수 서울에 왔으니까 조망간 슈퍼맨 제작진 섭외 들어가겠군요 ~^^  
2           아 기성용 몇마디 안하는데\r\n와중에 인범이 승우 챙기는거임 ㅜㅜ 졸멋탱이디  
3                                             진짜 훈훈하다ㅋㅋ  
4                                    구자철은 근데 잘하는 선수였나요?  
...                                                 ...  
2355                         기성용은 결국 명확한건 아직 없는 상황이네요 ㅠ  
2356                                다음 트랜스퍼H는 언제업로드하나요?  
2357  기성용 반시즌만뛰어달라고 들여왔는데 바이러스땜에 의문의 꽁돈만 날렸네ㅋㅋ\r\n서울...  
2358                                             서울은 삭제  
2359              빽패스는 이제 그만.....패스 성공률 100% 하지만 전부 빽패스  

[2360 rows x 3 columns]

In [4]:

#이모티콘 제거
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

#분석에 어긋나는 불용어구 제외 (특수문자, 의성어)
han = re.compile(r'[ㄱ-ㅎㅏ-ㅣ!?~,".\n\r#\ufeff\u200d]')

In [5]:
comment_list = []
for i in range(len(comment_data)):
    comment_list.append(comment_data['Comment'].iloc[i])

In [6]:
comment_result = []

for i in comment_list:
    tokens = re.sub(emoji_pattern,"",i)
    tokens = re.sub(han,"",tokens)
    comment_result.append(tokens)

comment_result = pd.DataFrame(comment_result, columns=["Comment"])

In [7]:
comment_result

Comment
0     자철 선수가 매개체 되어 해외에서 생활하는 선수들과 꾸준히 소통하고 상호 정보 공유...
1               성용 선수 서울에 왔으니까 조망간 슈퍼맨 제작진 섭외 들어가겠군요 ^^
2                 아 기성용 몇마디 안하는데와중에 인범이 승우 챙기는거임  졸멋탱이디
3                                               진짜 훈훈하다
4                                     구자철은 근데 잘하는 선수였나요
...                                                 ...
2355                          기성용은 결국 명확한건 아직 없는 상황이네요 
2356                                 다음 트랜스퍼H는 언제업로드하나요
2357  기성용 반시즌만뛰어달라고 들여왔는데 바이러스땜에 의문의 꽁돈만 날렸네서울의 거지같은...
2358                                             서울은 삭제
2359                   빽패스는 이제 그만패스 성공률 100% 하지만 전부 빽패스

[2360 rows x 1 columns]

## 형태소 분석기를 통한 명사 추출¶

In [8]:
!pip install konlpy

In [9]:
#한글 형태소 분석기인 konlpy사용 
#이때 konlpy에는 hannanum, kommoran, kkma, twitter이 있는데 해당 분석에서는 kkma사용
from konlpy.tag import Kkma
from konlpy.tag import Mecab
from konlpy.tag import Twitter
#apply lambda사용

In [10]:
from konlpy.tag import Twitter

def get_noun(comment_txt):
    twitter = Twitter()
    noun = []
    
    if len(comment_txt)>0:
        tw = twitter.pos(comment_txt)
        for i,j in tw:
            if j == 'Noun':
                noun.append(i)
    return noun
 
comment_result['token'] = comment_result['Comment'].apply(lambda x: get_noun(x))

C:\anaconda\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


JVMNotFoundException: No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.